In [1]:
%load_ext autoreload

In [2]:
%load_ext jupyter_black

In [3]:
%autoreload 2

In [4]:
import warnings

warnings.filterwarnings("ignore")

In [5]:
import os

# os.environ["TORCH_CPP_LOG_LEVEL"] = "INFO"
# os.environ["TORCH_DISTRIBUTED_DEBUG"] = "DETAIL"

In [6]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# The GPU id to use, "0" to  "7"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [7]:
import torch
import pytorch_lightning as pl

print(f"Torch={torch.__version__}, Lightening={pl.__version__}")

Torch=2.0.1+cu117, Lightening=2.0.6


In [8]:
from parrotletml import config

# from parrotletml.utils import seed_everything
# seed_everything()  # If you want deterministic behavior

### YOLODataset

#### DataModule

In [9]:
from parrotletml.datamodule import YOLODataModule

data_module = YOLODataModule(
    DATASET=config.DATASET,
    ANCHORS=config.ANCHORS,
    class_names=config.PASCAL_CLASSES,
    IMAGE_SIZE=config.IMAGE_SIZE,
    TRAIN_IMAGE_SIZES=config.TARGET_IMAGE_SIZE,
    S=config.SA,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    pin_memory=config.PIN_MEMORY,
    train_transforms=config.train_transforms,
    test_transforms=config.test_transforms,
)

#### Print Sample

In [10]:
# import random

# from parrotletml.utils import (
#     plot_couple_examples,
#     plot_image,
# )

# # S = config.S
# # anchors = config.ANCHORS
# # scaled_anchors = torch.tensor(anchors) / (
# #     1 / torch.tensor(S).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)
# # )

# number_of_samples = 5
# counter = 0

# data_module.setup("fit")

# # from parrotletml.model import YOLOv3

# # num_classes = 20
# # ym = YOLOv3(num_classes=num_classes)

# for x, y, S in data_module.train_dataloader()[0]:
#     counter = counter + 1
#     boxes = []

#     print(x.shape)
#     print(y[0].shape)

#     # out = ym(x[0].unsqueeze(0))

#     # print(out[0].shape)

#     break

#     anchors = config.ANCHORS
#     scaled_anchors = torch.tensor(anchors).unsqueeze(0) / (
#         1
#         / torch.stack(S)
#         .unsqueeze(1)
#         .unsqueeze(1)
#         .repeat(1, 3, 2, 1)
#         .permute(3, 0, 1, 2)
#     )

#     for i in range(y[0].shape[1]):
#         anchor = scaled_anchors[0][i]
#         boxes += cells_to_bboxes(y[i], is_preds=False, S=y[i].shape[2], anchors=anchor)[
#             0
#         ]
#     boxes = non_max_suppression(
#         boxes, iou_threshold=1, threshold=0.7, box_format="midpoint"
#     )

#     plot_image(x[0].permute(1, 2, 0).detach().to("cpu"), boxes)

#     if counter >= number_of_samples:
#         break

### PL YoloModel

#### YoloLoss

In [11]:
from parrotletml.loss import YoloLoss

#### Yolo MAP Calculation

In [12]:
from parrotletml.calculatemap import CalculateMAP

#### Yolo PL Module

In [13]:
from parrotletml.yoloplnet import LitYOLONet

model = LitYOLONet(
    device=config.DEVICE,
    lr=config.LEARNING_RATE,
)

## Training

In [15]:
import sys
from pytorch_lightning.callbacks import TQDMProgressBar


class MyProgressBar(TQDMProgressBar):
    def init_validation_tqdm(self):
        bar = super().init_validation_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

    def init_predict_tqdm(self):
        bar = super().init_predict_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

    # def init_test_tqdm(self):
    #     bar = super().init_test_tqdm()
    #     if not sys.stdout.isatty():
    #         bar.disable = True
    #     return bar


from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="yolov3")

from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# from pytorch_lightning.strategies import Fabric

# training
trainer = pl.Trainer(
    log_every_n_steps=1,
    callbacks=[
        MyProgressBar(refresh_rate=1),
        # RichProgressBar(refresh_rate=1, leave=False),
        LearningRateMonitor(logging_interval="epoch"),
        ModelCheckpoint(
            dirpath="ckpt_logs/yolov3",
            save_top_k=3,
            monitor="val_loss",
            mode="min",
            filename="model-{epoch:02d}-{val_loss:.2f}-{val_loss:4f}",
            save_last=True,
        ),
    ],
    logger=logger,
    precision=16,  # "16-mixed",
    accelerator="gpu",
    devices="auto",
    strategy="ddp_notebook",
    check_val_every_n_epoch=1,
    # limit_train_batches=5,
    # limit_val_batches=1,
    # limit_test_batches=1,
    max_epochs=config.NUM_EPOCHS * 2 // 5,
    # max_epochs=10,
)

# # ******************************

# from pytorch_lightning.tuner import Tuner

# tuner = Tuner(trainer)

# ## Find Batch Size *************

# batch_size = tuner.scale_batch_size(model, datamodule=data_module, mode="binsearch")

# print(batch_size)

# # Found GPU can handle 132 Batch Size will use 128

# ## Find LR *********************

# lr_finder = tuner.lr_find(model, datamodule=data_module, num_training=200)

# # Results can be found in
# print(lr_finder.results)

# # Plot with
# fig = lr_finder.plot(suggest=True)
# fig.show()

# # Pick point based on plot, or get a suggestion
# new_lr = lr_finder.suggestion()

# # update hparams of the model
# model.hparams.lr = new_lr

# # ******************************

# Uncomment the following line to train the model
trainer.fit(model, datamodule=data_module)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:lightning_fabric.utilities.distributed:Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
INFO:pytorch_lightning.utilities.rank_zero:----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]
INFO:pytorch_lightning.utilities.rank_zero:Loading `train_dataloader` to 

Epoch 39: 100%|██████████| 518/518 [06:20<00:00,  1.36it/s, v_num=314, val_loss=2.640, val_class_accuracy=91.80, val_no_obj_accuracy=95.90, val_obj_accuracy=95.50, train_loss=4.640, train_class_accuracy=91.50, train_no_obj_accuracy=94.00, train_obj_accuracy=97.40]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=40` reached.


Epoch 39: 100%|██████████| 518/518 [06:27<00:00,  1.34it/s, v_num=314, val_loss=2.640, val_class_accuracy=91.80, val_no_obj_accuracy=95.90, val_obj_accuracy=95.50, train_loss=4.640, train_class_accuracy=91.50, train_no_obj_accuracy=94.00, train_obj_accuracy=97.40]


In [18]:
trainer.test(
    model,
    # dataloaders=val_dataloader,
    datamodule=data_module,
    # ckpt_path="ckpt_logs/yolov3/last.ckpt",
)

INFO:lightning_fabric.utilities.distributed:Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
INFO:pytorch_lightning.utilities.rank_zero:----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Testing DataLoader 0: 100%|██████████| 518/518 [31:37<00:00,  3.66s/it]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         map_epoch         │    0.7876859903335571     │
│    test_class_accuracy    │     96.89830017089844     │
│   test_no_obj_accuracy    │     96.0931396484375      │
│     test_obj_accuracy     │     97.3210678100586      │
└───────────────────────────┴───────────────────────────┘

[{'map_epoch': 0.7876859903335571,
  'test_class_accuracy': 96.89830017089844,
  'test_no_obj_accuracy': 96.0931396484375,
  'test_obj_accuracy': 97.3210678100586}]

### Load Model from CHECKPOINT

In [1]:
from parrotletml.yoloplnet import LitYOLONet

#### Load Model From CheckPoint

In [2]:
loaded_model = LitYOLONet.load_from_checkpoint("ckpt_logs/yolov3/last.ckpt")

# final-model-epoch=39-val_loss=3.16-val_loss=3.162749.ckpt
# final-model-epoch=39-val_loss=3.18-val_loss=3.178922.ckpt ## Multi Image Size 10, 13
# final-model-epoch=39-val_loss=2.27-val_loss=2.265536.ckpt

loaded_model.eval()

print("Loaded Checkpoint")

Loaded Checkpoint


i#### Save Model to Multiple Formats

In [3]:


loaded_model.to_onnx(
    "ckpt_logs/yolov3/yolov3.onnx",
    input_sample=torch.empty(1, 3, 416, 416, dtype=torch.float32),
    export_params=True,
    input_names=["image"],
    output_names=["classes"],
)

torch.save(loaded_model.yolo.state_dict(), "ckpt_logs/yolov3/yolov3.pth")

NameError: name 'torch' is not defined

#### Load Model in ONNX and PT

In [ ]:
import cv2
from dataclasses import dataclass

@dataclass
class config:
    DEVICE = "cuda"  # "cuda" if torch.cuda.is_available() else "cpu"

    mean = [0.4914, 0.4822, 0.4465]
    std = [0.2470, 0.2435, 0.2616]

    IMAGE_SIZE = 416
    
    ANCHORS = [
        [(0.28, 0.22), (0.38, 0.48), (0.9, 0.78)],
        [(0.07, 0.15), (0.15, 0.11), (0.14, 0.29)],
        [(0.02, 0.03), (0.04, 0.07), (0.08, 0.06)],
    ]  # Note these have been rescaled to be between [0, 1]
    S = [IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8]

    NUM_CLASSES = 20

    PASCAL_CLASSES = [
        "aeroplane",
        "bicycle",
        "bird",
        "boat",
        "bottle",
        "bus",
        "car",
        "cat",
        "chair",
        "cow",
        "diningtable",
        "dog",
        "horse",
        "motorbike",
        "person",
        "pottedplant",
        "sheep",
        "sofa",
        "train",
        "tvmonitor",
    ]

In [4]:
import onnxruntime as ort
from parrotletml.model import YOLOv3

# Load the ONNX model
onnxmodel = ort.InferenceSession("ckpt_logs/yolov3/yolov3.onnx")

# Load the PT Model
ptmodel = YOLOv3(num_classes=config.NUM_CLASSES).to(config.DEVICE)
ptmodel.load_state_dict(
    torch.load("ckpt_logs/yolov3/yolov3.pth", map_location=config.DEVICE)
)
ptmodel.eval()

print("Loaded")

NameError: name 'config' is not defined

### Inferencing

In [ ]:
import cv2
import torch

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from typing import List

from pytorch_grad_cam.utils.image import show_cam_on_image
from parrotletml.mygradcam import YoloCAM
from parrotletml.utils import cells_to_bboxes, overlay_predictions, non_max_suppression

import glob

from parrotletml.utils import resize_image, normalize_image


scaled_anchors = torch.tensor(config.ANCHORS, device=config.DEVICE) * torch.tensor(
    config.S, device=config.DEVICE
).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)

cam = YoloCAM(model=ptmodel, target_layers=[ptmodel.layers[-3]], use_cuda=False)


# Define the preprocessing function for the input image
def preprocess_image(image):
    # # Read the image using OpenCV and convert to RGB
    # image = cv2.cvtColor(cv2.imread(impath, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)

    image = np.asarray(image)

    # Resize, normalize, and convert image to the required format
    image = resize_image(image, (416, 416), fill=(0, 0, 0))

    # image = np.expand_dims(normalize_cifar_cv_image(resize_image(impath, (32, 32)).transpose(2, 0, 1)).astype(np.float32), axis=0)
    # image = np.expand_dims(
    #     normalize_image(image, mean=[0, 0, 0], std=[1, 1, 1])
    #     .transpose(2, 0, 1)
    #     .astype(np.float32),
    #     axis=0,
    # )

    image = normalize_image(image, mean=[0, 0, 0], std=[1, 1, 1]).astype(np.float32)

    return image, torch.from_numpy(image.transpose(2, 0, 1))


@torch.inference_mode()
def predict(
    image: np.ndarray,
    iou_thresh: float = 0.5,
    thresh: float = 0.4,
    show_cam: bool = True,
    transparency: float = 0.3,
) -> List[np.ndarray]:
    # transformed_image = torch.tensor(
    #     config.transforms(image=image)["image"].unsqueeze(0), device=config.DEVICE
    # )

    # print(transformed_image.shape)

    image, transformed_image = preprocess_image(image)
    transformed_image = torch.tensor(transformed_image, device=config.DEVICE).unsqueeze(
        0
    )
    # print(transformed_image.shape)

    output = ptmodel(transformed_image)

    bboxes = [[] for _ in range(1)]
    for i in range(3):
        batch_size, A, S, _, _ = output[i].shape
        anchor = scaled_anchors[i]
        boxes_scale_i = cells_to_bboxes(
            torch.tensor(output[i], device=config.DEVICE), anchor, S=S, is_preds=True
        )
        for idx, (box) in enumerate(boxes_scale_i):
            bboxes[idx] += box

    nms_boxes = non_max_suppression(
        bboxes[0],
        iou_threshold=iou_thresh,
        threshold=thresh,
        box_format="midpoint",
    )
    plot_img = np.clip(
        overlay_predictions(
            image,
            nms_boxes,
            class_labels=config.PASCAL_CLASSES,
            confidence_threshold=0.75,
        ),
        0,
        1,
    )
    if not show_cam:
        return [plot_img, image]

    grayscale_cam = cam(transformed_image, scaled_anchors)[0, :, :]
    # img = cv2.resize(image, (416, 416))
    # img = np.float32(img) / 255
    cam_image = show_cam_on_image(
        image,
        grayscale_cam,
        use_rgb=True,
        image_weight=transparency,
    )
    return [plot_img, image, cam_image]


import glob

if __name__ == "__main__":
    for p in glob.glob("samples/*.*g"):
        img_path = p  # "samples/ammar.jpg"
        image = np.array(Image.open(img_path))

        # image = resize_image(image, (416, 416), fill=(0, 0, 0))

        pimage = predict(image)

        # Create a figure with two subplots
        fig, (ax2, ax3) = plt.subplots(
            1, 2, figsize=(7, 10)
        )  # Adjust fig size as needed

        # Plot the images in the subplots
        # ax1.imshow(image)
        # ax1.set_title("Original Image")
        # ax1.axis("off")  # Turn off-axis labels and ticks

        # Plot the images in the subplots
        ax2.imshow(pimage[0])
        ax2.set_title("Overlayed Image")
        ax2.axis("off")  # Turn off-axis labels and ticks

        ax3.imshow(pimage[2])
        ax3.set_title("GradCam")
        ax3.axis("off")  # Turn off-axis labels and ticks

        # ax4.imshow(pimage[1])
        # ax4.set_title("Transformed")
        # ax4.axis("off")  # Turn off-axis labels and ticks

        # Display the figure
        plt.tight_layout()  # Adjust layout for better spacing
        plt.show()

    # cv2.imshow("image", image[0])